Perform NicheNet analysis
================
Maria Solovey
2022-- DATE

In this notebook, we perform a basic NicheNet analysis on a Seurat object. We start by creating a Seurat object from the normalized count matrix and cell annotation file. You can find the notebooks to preprocessing, filtering and UMAP as well as the details of the input dataset in XXX. 


The used NicheNet custom networks, ligand-target matrix and expression
data of interacting cells can be downloaded from Zenodo. The NicheNet
networks and ligand-target matrix at [![DOI](https://zenodo.org/badge/DOI/10.5281/zenodo.3260758.svg)](https://doi.org/10.5281/zenodo.3260758)
and the raw single-cell data along with normalized and filtered at [![DOI](https://zenodo.org/badge/DOI/10.5281/zenodo.3531889.svg)](https://doi.org/10.5281/zenodo.3531889)


This pipeline is adapted from the offical tutorial of NicheNet's on their GitHub page. 

[Perform NicheNet analysis starting from a Seurat object: step-by-step analysis
](https://github.com/saeyslab/nichenetr/blob/master/vignettes/seurat_steps.md)


In [ ]:
# notebook.output.textLineLimit=30

# Prepare NicheNet analysis

## Load required packages, read in the normalized count matrix of interacting cells and custom build of ligand-target prior model for NicheNet, ligand-receptor network and weighted integrated networks.

To identify potential ligand-receptor interactions between two cell populations, the NicheNet approach requires both the NicheNet ligand-receptor network and weighted networks. The ligand-target matrix provides information on the potential of certain ligands to regulate the expression of specific target genes, enabling prioritization of possible interactions based on observed gene expression effects through NicheNet's ligand activity analysis. This prioritization helps to infer the affected target genes of the selected ligands.

In [1]:
# libraries
library(nichenetr)
library(ggplot2)
library(Seurat)
library(tidyverse)
library(igraph)
require(circlize)
library(R.utils)
library(data.table) #to read gz file

Attaching SeuratObject

── Attaching core tidyverse packages ──────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────── tidyverse 2.0.0 ──
✔ dplyr     1.1.4     ✔ readr     2.1.5
✔ forcats   1.0.0     ✔ stringr   1.5.1
✔ lubridate 1.9.3     ✔ tibble    3.2.1
✔ purrr     1.0.2     ✔ tidyr     1.3.1
── Conflicts ────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
ℹ Use the conflicted package (<http://conflicted.r-lib.org/>) to force all conflicts to become errors

Attaching package: ‘igraph’


The following objects are masked from ‘package:lubridate’:

    %--%, union


The following objects are masked from ‘package:dplyr’:

    as_data_frame, groups, union


The following objects are m

In [3]:
sessionInfo()

R version 4.3.2 (2023-10-31)
Platform: x86_64-conda-linux-gnu (64-bit)
Running under: CentOS Linux 7 (Core)

Matrix products: default
BLAS/LAPACK: /work/project/ladcol_011/conda_path/miniconda3/envs/NicheNet/lib/libopenblasp-r0.3.26.so;  LAPACK version 3.12.0

locale:
 [1] LC_CTYPE=en_US.UTF-8       LC_NUMERIC=C              
 [3] LC_TIME=de_DE.UTF-8        LC_COLLATE=en_US.UTF-8    
 [5] LC_MONETARY=de_DE.UTF-8    LC_MESSAGES=en_US.UTF-8   
 [7] LC_PAPER=de_DE.UTF-8       LC_NAME=C                 
 [9] LC_ADDRESS=C               LC_TELEPHONE=C            
[11] LC_MEASUREMENT=de_DE.UTF-8 LC_IDENTIFICATION=C       

time zone: Europe/Berlin
tzcode source: system (glibc)

attached base packages:
[1] stats     graphics  grDevices utils     datasets  methods   base     

other attached packages:
 [1] data.table_1.14.10 R.utils_2.12.3     R.oo_1.26.0        R.methodsS3_1.8.2 
 [5] circlize_0.4.15    igraph_1.6.0       lubridate_1.9.3    forcats_1.0.0     
 [9] stringr_1.5.1      dplyr_1.1.

### Read in the expression data of interacting cells:
The dataset used here is publicly available single-cell data from XXX. The data was processed, and filtered by applying XXX. 

In [7]:
input_dir <- "../../../../../data_preprocessing/Lasry/2.filtering/outs/"
db_dir <- "../build_customDB/NNET_Custom/"
output_dir <- "outs/"

In [11]:
# # load counts
# print("load counts")
# counts <- read.table(gzfile(paste0(path_in,"/counts_corr.csv.gz")
#                             )
#                      ,sep = ","
#                      ,row.names = 1
#                      ,header = TRUE
#                      )
# # load counts

counts <- fread(paste0(input_dir,"counts_norm.csv.gz"), header = TRUE,check.names=FALSE)
counts <- as.data.frame(counts)
rownames(counts) <- counts$gene_symbol
counts <- counts[,-1]
head(str(counts))
print(str(counts))

'data.frame':	15770 obs. of  46702 variables:
 $ X2020.09.15.AML0024.CATCAAGGTTAGCGGA           : num  0 0 0 0 0 0 0 0 0 0 ...
 $ X2020.09.15.AML0024.CATCAAGTCCGAGAAG           : num  0 0 0 0 0 0 0 0 0 0 ...
 $ X2020.09.15.AML0024.CATCCACAGGGACCAT           : num  0 0 0 0 0 ...
 $ X2020.09.15.AML0024.CCTCAACAGAGCAAGA           : num  0 0 0 0 0 0 0 0 0 0 ...
 $ X2020.09.15.AML0024.CCTCAACAGTTCCATG           : num  0 0.993 0 0 0 ...
 $ X2020.09.15.AML0024.CCTCAACGTAGAATAC           : num  0 0 0 0 0 0 0 0 0 0 ...
 $ X2020.09.15.AML0024.CCTCAACGTTCTCCCA           : num  0 0 0 0 0 0 0 0 0 0 ...
 $ X2020.09.15.AML0024.CCTCAACTCCGAACGC           : num  0 0 0 0 0 ...
 $ X2020.09.15.AML0024.CCTCAACTCTAGTCAG           : num  0 0 0 0 0 0 0 0 0 0 ...
 $ X2020.09.15.AML0024.CCTCACAAGACAGTCG           : num  0 0 0 0 0 0 0 0 0 0 ...
 $ X2020.09.15.AML0024.CCTCACACAATTGCCA           : num  0 0 0 0 0 0 0 0 0 0 ...
 $ X2020.09.15.AML0024.CCTCACACAGAACTAA           : num  0 0 0 0 0 0 0 0 0 0 ...
 $ X2020

NULL

'data.frame':	15770 obs. of  46702 variables:
 $ X2020.09.15.AML0024.CATCAAGGTTAGCGGA           : num  0 0 0 0 0 0 0 0 0 0 ...
 $ X2020.09.15.AML0024.CATCAAGTCCGAGAAG           : num  0 0 0 0 0 0 0 0 0 0 ...
 $ X2020.09.15.AML0024.CATCCACAGGGACCAT           : num  0 0 0 0 0 ...
 $ X2020.09.15.AML0024.CCTCAACAGAGCAAGA           : num  0 0 0 0 0 0 0 0 0 0 ...
 $ X2020.09.15.AML0024.CCTCAACAGTTCCATG           : num  0 0.993 0 0 0 ...
 $ X2020.09.15.AML0024.CCTCAACGTAGAATAC           : num  0 0 0 0 0 0 0 0 0 0 ...
 $ X2020.09.15.AML0024.CCTCAACGTTCTCCCA           : num  0 0 0 0 0 0 0 0 0 0 ...
 $ X2020.09.15.AML0024.CCTCAACTCCGAACGC           : num  0 0 0 0 0 ...
 $ X2020.09.15.AML0024.CCTCAACTCTAGTCAG           : num  0 0 0 0 0 0 0 0 0 0 ...
 $ X2020.09.15.AML0024.CCTCACAAGACAGTCG           : num  0 0 0 0 0 0 0 0 0 0 ...
 $ X2020.09.15.AML0024.CCTCACACAATTGCCA           : num  0 0 0 0 0 0 0 0 0 0 ...
 $ X2020.09.15.AML0024.CCTCACACAGAACTAA           : num  0 0 0 0 0 0 0 0 0 0 ...
 $ X2020

In [14]:
anno_cells <- read.table(paste0(input_dir,"anno_cells_norm.txt")
                         ,sep = "\t"
                         ,header = TRUE
                         )

In [16]:
row.names(anno_cells) <- anno_cells$cell

In [26]:
colnames(counts) <- row.names(anno_cells)

In [27]:
head(colnames(counts))

[1] "2020-09-15-AML0024:CATCAAGGTTAGCGGA" "2020-09-15-AML0024:CATCAAGTCCGAGAAG"
[3] "2020-09-15-AML0024:CATCCACAGGGACCAT" "2020-09-15-AML0024:CCTCAACAGAGCAAGA"
[5] "2020-09-15-AML0024:CCTCAACAGTTCCATG" "2020-09-15-AML0024:CCTCAACGTAGAATAC"

In [28]:
head(rownames(anno_cells))

[1] "2020-09-15-AML0024:CATCAAGGTTAGCGGA" "2020-09-15-AML0024:CATCAAGTCCGAGAAG"
[3] "2020-09-15-AML0024:CATCCACAGGGACCAT" "2020-09-15-AML0024:CCTCAACAGAGCAAGA"
[5] "2020-09-15-AML0024:CCTCAACAGTTCCATG" "2020-09-15-AML0024:CCTCAACGTAGAATAC"

In [29]:
# get average expr 
avg_expression <- rowMeans(counts)
gene_names <- rownames(counts)
avg_expr_table <- data.frame(Gene = gene_names, OverallAverageExpression = avg_expression)

In [30]:
# get average expr within cohorts
case <- anno_cells$health_status == "AML"
case_idx <- rownames(anno_cells)[case]
avg_expression_case <- rowMeans(counts[,case_idx])


control <- anno_cells$health_status == "healthy"
control_idx <- rownames(anno_cells)[control]
avg_expression_control <- rowMeans(counts[,control_idx])

In [31]:
# average expr table
avg_expr_table$case_avg <- avg_expression_case
avg_expr_table$control_avg <- avg_expression_control
write_csv2(avg_expr_table, paste0(output_dir,"Lasry_avg_expr_table.csv"))

In [32]:
avg_expr_table

,Gene,OverallAverageExpression,case_avg,control_avg
,<chr>,<dbl>,<dbl>,<dbl>
AL627309.5,AL627309.5,0.009435514,0.009542306,0.0093458831
LINC01409,LINC01409,0.057288910,0.058816451,0.0560068250
LINC01128,LINC01128,0.042966343,0.044750354,0.0414689997
LINC00115,LINC00115,0.012024620,0.011667666,0.0123242164
FAM41C,FAM41C,0.017675162,0.020741900,0.0151012084
AL645608.2,AL645608.2,0.001801450,0.003286857,0.0005547284
NOC2L,NOC2L,0.140301112,0.128436107,0.1502595668
KLHL17,KLHL17,0.010874237,0.014114589,0.0081545675
PLEKHN1,PLEKHN1,0.006620852,0.007515589,0.0058698871


In [33]:
srt=CreateSeuratObject(counts=counts, meta.data=anno_cells)

#### Read in ligand-target prior model, ligand-receptor network and weighted integrated networks:

In [34]:
ligand_target_matrix = readRDS(paste0(db_dir,"ligand_target_matrixWithweights.rds"))
lr_network = readRDS(paste0(db_dir,"lig_rec_sources.rds"))
weighted_networks = readRDS(paste0(db_dir,"weighted_networksWithSourceWeights.rds"))
weighted_networks_lr = weighted_networks$lr_sig %>% inner_join(lr_network %>% distinct(from,to), by = c("from","to"))
# weighted_networks_lr = weighted_networks_lr %>% filter(from %in% lr_network$from & to %in% lr_network$to)


In [35]:
srt@meta.data$health_status %>% table()

.
    AML healthy 
  21311   25391 

In [36]:
srt@meta.data$cell_type %>% table()

.
    B    DC   Ery  Gran  HSPC  Mono    NK     T 
 4765  1634  1674  2332  3169 18004  3078 12046 

In [37]:
Idents(srt) <- "cell_type"

In [38]:
cell_type_list <- unique(srt@meta.data$cell_type)
sample_list <- unique(srt@meta.data$sample_ID)

### 1. Define a “sender/niche” cell population and a “receiver/target”
In this case study, we consider each cell type as reciever as well as a sender in a for loop. We consider a gene to be expressed when it is expressed in at least 10% of cells in one cluster. We have 8 cell types, and each of them is tested as a sender as well as reciever. This allows us to check the cell types of sender and reciever later on during the interpretation. 

### 2. Define a gene set of interest
Here, the gene set of interest are the genes differentially expressed in each cell type after --AML diagnosis--. Thus, `AML` is the condition of interest, whereas the reference condition is annotated as `healthy`. The method to calculate the differential expression is here the standard Seurat Wilcoxon test. 

### 3. Define a set of potential ligands
These are ligands that are expressed by the sender cell population and bind a receptor expressed by the reciever population.

### 4. Perform NicheNet ligand activity analysis

Once NicheNet outputs which ligands from one or more cell population(s) (“sender/niche”) are most likely to affect target gene expression in an interacting cell population (“receiver/target”), we extract the pairs from the `weighted_networks_lr`

In [39]:
# Create an empty list to store the expressed genes for each sample and cell type
expressed_genes_list <- list()

# Iterate through each sample in the sample_list
for (sample in sample_list){
    
    # Subset the Seurat object to the current sample
    sample_subset <- subset(x = srt, subset = sample_ID == sample)
    
    # Get a list of unique cell types in the current sample
    cell_types <- unique(sample_subset@meta.data$cell_type)
    
    # Iterate through each cell type in the current sample
    for (cell_type in cell_types){
        
        # Call the function 'get_expressed_genes' to get the genes that are expressed in the current cell type
        expressed_genes <- get_expressed_genes(cell_type, sample_subset, 0.10)
        
        # Convert the expressed genes to a vector and remove duplicates
        expressed_genes <- expressed_genes %>% unlist() %>% unique()
        
        # Add the list of expressed genes for the current sample and cell type to the 'expressed_genes_list'
        expressed_genes_list[[sample]][[cell_type]] <- expressed_genes
        
    }
}


In [40]:
# total=0
# for (sample in sample_list){
    
#     for (cell_type in cell_type_list){
# #         print(sample)
# #         print(cell_type)
# #         print(length(expressed_genes_list[[sample]][[cell_type]]))
        
#         total=total+length(expressed_genes_list[[sample]][[cell_type]])
#     }
# }

# print(total)

[1] 484944


In [41]:
sample_subset

An object of class Seurat 
15770 features across 3874 samples within 1 assay 
Active assay: RNA (15770 features, 0 variable features)

In [42]:
length(expressed_genes_list[[sample]][[cell_type]])

[1] 3580

In [43]:
# Create an empty list to store the differential expression tables for each cell type
DE_tables <- list()

# Iterate through each cell type in the cell_type_list
for (cell_type in cell_type_list){
    
    # Subset the Seurat object to the current cell type
    seurat_subset <- subset(srt, idents = cell_type)
    
    # Set the identity of the cells in the subset to the 'health_status' column
    seurat_subset <- SetIdent(seurat_subset, value = seurat_subset[["health_status"]])
    
    # Set the conditions of interest and reference for the differential expression analysis
    condition_oi <- "AML"
    condition_reference <- "healthy"
    
    # Call the function 'FindMarkers' to perform differential expression analysis and get the differential expression table
    DE_table <- FindMarkers(object = seurat_subset, ident.1 = condition_oi, 
                                    ident.2 = condition_reference, min.pct = 0.10) %>% rownames_to_column("gene")
    
    # Add the differential expression table for the current cell type to the 'DE_tables' list
    DE_tables[[cell_type]] <- DE_table
}


This function is designed to transform a data frame into a new data frame in wide format, with the same structure as the output from a community tool. The reason for this is to ensure consistency in the structure of the results table, which can facilitate further analysis or comparison with other data sets.

In particular, the input data frame is expected to have columns named "interaction", "sample", and "weight", representing interactions between entities in different samples. The function pivots the data frame from long format to wide format, with "interaction" as the ID column, "sample" as the column names, and "weight" as the values.

The resulting data frame has the same number of rows as the input data frame, but the number of columns is equal to the number of unique values in the "sample" column. This structure is consistent with the output of a community tool, which typically provides a table of interactions with each sample represented as a separate column.

In [45]:
# Define a function 'transform_interaction_df' that takes a data frame as input
transform_interaction_df <- function(df) {
  
  # Pivot the data frame to wide format, with 'interaction' as the ID column, 
  # 'sample' as the column names, and 'weight' as the values
  df_new <- df %>%
    pivot_wider(
      id_cols = interaction_ID,
      names_from = sample,
      values_from = c("weight")
    ) %>%
    
    # Reorder the columns to have 'interaction' as the first column
    select(interaction_ID, everything()) 
  
  # Return the pivoted data frame
  return(df_new)
}


In [47]:
# Get unique ligands and receptors from lr_network

ligands_in_db = lr_network %>% pull(from) %>% unique()
receptors_in_db = lr_network %>% pull(to) %>% unique()

# Initialize an empty list to store results
results=list()
ligand_activities_list <- list()
lr_network_df_large_full <- list()


for (receiver in cell_type_list){
    
    # Get DE genes for the receiver cell type
    geneset_oi <- DE_tables[[receiver]] %>% filter(p_val_adj <= 0.05 & abs(avg_log2FC) >= 0.25) %>% pull(gene)
    
#     n_DE_genes <- length(geneset_oi)
    
    # Iterate over each sample
    for (sample in names(expressed_genes_list)){
        
        # Get expressed genes for the receiver cell type in the current sample
        expressed_genes_receiver <- expressed_genes_list[[sample]][[receiver]]
        
        
        # Get the intersection of DE genes and expressed genes for the receiver cell type in the current sample
        geneset_oi_sample_expressed <- geneset_oi[geneset_oi %in% expressed_genes_receiver]
        
        # Get the intersection of DE genes and ligand-target gene matrix rows
        geneset_oi_sample_expressed <- geneset_oi_sample_expressed[geneset_oi_sample_expressed %in% colnames(ligand_target_matrix)]
        
#         geneset_oi_sample_expressed <- geneset_oi_sample_expressed[geneset_oi_sample_expressed %in% lr_network$from]
        
        
        
        # Get the intersection of expressed genes for the receiver cell type in the current sample and ligand-target gene matrix rows
        background_expressed_genes  <- expressed_genes_receiver[expressed_genes_receiver %in% rownames(ligand_target_matrix)]
        
        # Iterate over each sender cell type
        for (sender in names(expressed_genes_list[[sample]])){
            
            
#             de_rec <- geneset_oi
#             de_lig <- DE_tables[[sender]] %>% filter(p_val_adj <= 0.05 & abs(avg_log2FC) >= 0.25) %>% pull(gene)
            
            
            # Get expressed genes for the sender cell type in the current sample
            expressed_genes_sender <- expressed_genes_list[[sample]][[sender]]
            
            # Check if the sender cell type is not null for the receiver cell type
            if (!is.null(expressed_genes_list[[sample]][[receiver]])){
                
                # Get the intersection of expressed ligands and ligands in the lr_network
                expressed_ligands <- expressed_genes_sender[expressed_genes_sender %in% ligands_in_db]
                expressed_receptors <- expressed_genes_receiver[expressed_genes_receiver %in% receptors_in_db]
                
                
                
                # Get the potential ligands based on the expressed ligands and receptors
                potential_ligands = lr_network %>% 
                    filter(from %in% expressed_ligands & to %in% expressed_receptors) %>% 
                        pull(from) %>% unique()
                
                
                
                potential_ligands = potential_ligands[potential_ligands %in% colnames(ligand_target_matrix)]
                
                
                # Predict ligand activities with NicheNet
                ligand_activities = predict_ligand_activities(geneset = geneset_oi_sample_expressed, 
                                                              background_expressed_genes = background_expressed_genes, 
                                                              ligand_target_matrix = ligand_target_matrix, 
                                                              potential_ligands = potential_ligands)
                
                # Arrange ligand activities by pearson correlation coefficient and add a rank column
                ligand_activities = ligand_activities %>% arrange(-pearson) %>% mutate(rank = rank(desc(pearson)))
                
                
                # Get the potential ligands based on the ligand activities
                potential_ligands = ligand_activities %>% pull(test_ligand) %>% unique()
                
#                 potential_ligands = intersect(potential_ligands, de_lig)
                
                
#                 best_upstream_ligands = ligand_activities %>% top_n(20, pearson) %>% arrange(-pearson) %>% pull(test_ligand) %>% unique()

                
                ligand_activities = ligand_activities %>% arrange(-aupr_corrected) %>% mutate(rank = rank(desc(aupr_corrected)))
                
                best_upstream_ligands = ligand_activities %>% top_n(30, aupr_corrected) %>% arrange(-aupr_corrected) %>% pull(test_ligand) %>% unique()
                
                # Filter the lr_network to only include interactions between potential ligands and expressed receptors
                lr_network_expressed = lr_network %>% 
                    filter(from %in% best_upstream_ligands & to %in% expressed_receptors) %>% distinct(from,to)
                
                
                # Get the corresponding interactions from the weighted network
                lr_network_df_large = weighted_networks_lr %>% filter(from %in% best_upstream_ligands & 
                                                                      to %in% expressed_receptors)
                
                # Add metadata to the network dataframe
                lr_network_df_large$sample <- sample
                lr_network_df_large$cell_type_l <- sender
                lr_network_df_large$cell_type_r <- receiver
                lr_network_df_large$pair <- paste(lr_network_df_large$from,lr_network_df_large$to, sep="_")
                lr_network_df_large$interaction_ID = paste0(lr_network_df_large$cell_type_l,":",
                                                         lr_network_df_large$from,"_",
                                                         lr_network_df_large$cell_type_r,":",
                                                         lr_network_df_large$to)
                
                
                df_new <- transform_interaction_df(lr_network_df_large)
                results[[sample]] <- rbind(results[[sample]],df_new)
                ligand_activities_list[[sample]][[sender]][[receiver]] <- ligand_activities
                
                lr_network_df_large_full[[sample]][[sender]][[receiver]] <- lr_network_df_large
#                 print(receiver)
#                 print(nrow(lr_network_df_large))
#                 write.csv(ligand_activities, paste("results6/",sample,"_",sender,"_",receiver,"_ligand_activities.csv", sep=""))
#                 write.csv(lr_network_df_large, paste("results6/",sample,"_",sender,"_",receiver,"_LR.csv", sep=""))
            }
        }
        
    }
    
}

Warning message in evaluate_target_prediction(setting, ligand_target_matrix, ligands_position):
“all target gene probability score predictions have same value”
Warning message in cor(prediction, response):
“the standard deviation is zero”
Warning message in cor(prediction, response, method = "s"):
“the standard deviation is zero”
Warning message in evaluate_target_prediction(setting, ligand_target_matrix, ligands_position):
“all target gene probability score predictions have same value”
Warning message in cor(prediction, response):
“the standard deviation is zero”
Warning message in cor(prediction, response, method = "s"):
“the standard deviation is zero”
Warning message in evaluate_target_prediction(setting, ligand_target_matrix, ligands_position):
“all target gene probability score predictions have same value”
Warning message in cor(prediction, response):
“the standard deviation is zero”
Warning message in cor(prediction, response, method = "s"):
“the standard deviation is zero”
Warn

Warning message in cor(prediction, response):
“the standard deviation is zero”
Warning message in cor(prediction, response, method = "s"):
“the standard deviation is zero”
Warning message in evaluate_target_prediction(setting, ligand_target_matrix, ligands_position):
“all target gene probability score predictions have same value”
Warning message in cor(prediction, response):
“the standard deviation is zero”
Warning message in cor(prediction, response, method = "s"):
“the standard deviation is zero”
Warning message in evaluate_target_prediction(setting, ligand_target_matrix, ligands_position):
“all target gene probability score predictions have same value”
Warning message in cor(prediction, response):
“the standard deviation is zero”
Warning message in cor(prediction, response, method = "s"):
“the standard deviation is zero”
Warning message in evaluate_target_prediction(setting, ligand_target_matrix, ligands_position):
“all target gene probability score predictions have same value”
Warn

Warning message in cor(prediction, response):
“the standard deviation is zero”
Warning message in cor(prediction, response, method = "s"):
“the standard deviation is zero”
Warning message in evaluate_target_prediction(setting, ligand_target_matrix, ligands_position):
“all target gene probability score predictions have same value”
Warning message in cor(prediction, response):
“the standard deviation is zero”
Warning message in cor(prediction, response, method = "s"):
“the standard deviation is zero”
Warning message in evaluate_target_prediction(setting, ligand_target_matrix, ligands_position):
“all target gene probability score predictions have same value”
Warning message in cor(prediction, response):
“the standard deviation is zero”
Warning message in cor(prediction, response, method = "s"):
“the standard deviation is zero”
Warning message in evaluate_target_prediction(setting, ligand_target_matrix, ligands_position):
“all target gene probability score predictions have same value”
Warn

Warning message in cor(prediction, response):
“the standard deviation is zero”
Warning message in cor(prediction, response, method = "s"):
“the standard deviation is zero”
Warning message in evaluate_target_prediction(setting, ligand_target_matrix, ligands_position):
“all target gene probability score predictions have same value”
Warning message in cor(prediction, response):
“the standard deviation is zero”
Warning message in cor(prediction, response, method = "s"):
“the standard deviation is zero”
Warning message in evaluate_target_prediction(setting, ligand_target_matrix, ligands_position):
“all target gene probability score predictions have same value”
Warning message in cor(prediction, response):
“the standard deviation is zero”
Warning message in cor(prediction, response, method = "s"):
“the standard deviation is zero”
Warning message in evaluate_target_prediction(setting, ligand_target_matrix, ligands_position):
“all target gene probability score predictions have same value”
Warn

Warning message in cor(prediction, response):
“the standard deviation is zero”
Warning message in cor(prediction, response, method = "s"):
“the standard deviation is zero”
Warning message in evaluate_target_prediction(setting, ligand_target_matrix, ligands_position):
“all target gene probability score predictions have same value”
Warning message in cor(prediction, response):
“the standard deviation is zero”
Warning message in cor(prediction, response, method = "s"):
“the standard deviation is zero”
Warning message in evaluate_target_prediction(setting, ligand_target_matrix, ligands_position):
“all target gene probability score predictions have same value”
Warning message in cor(prediction, response):
“the standard deviation is zero”
Warning message in cor(prediction, response, method = "s"):
“the standard deviation is zero”
Warning message in evaluate_target_prediction(setting, ligand_target_matrix, ligands_position):
“all target gene probability score predictions have same value”
Warn

...

Warning message in cor(prediction, response):
“the standard deviation is zero”
Warning message in cor(prediction, response, method = "s"):
“the standard deviation is zero”
Warning message in evaluate_target_prediction(setting, ligand_target_matrix, ligands_position):
“all target gene probability score predictions have same value”
Warning message in cor(prediction, response):
“the standard deviation is zero”
Warning message in cor(prediction, response, method = "s"):
“the standard deviation is zero”
Warning message in evaluate_target_prediction(setting, ligand_target_matrix, ligands_position):
“all target gene probability score predictions have same value”
Warning message in cor(prediction, response):
“the standard deviation is zero”
Warning message in cor(prediction, response, method = "s"):
“the standard deviation is zero”
Warning message in evaluate_target_prediction(setting, ligand_target_matrix, ligands_position):
“all target gene probability score predictions have same value”
Warn

Warning message in cor(prediction, response):
“the standard deviation is zero”
Warning message in cor(prediction, response, method = "s"):
“the standard deviation is zero”
Warning message in evaluate_target_prediction(setting, ligand_target_matrix, ligands_position):
“all target gene probability score predictions have same value”
Warning message in cor(prediction, response):
“the standard deviation is zero”
Warning message in cor(prediction, response, method = "s"):
“the standard deviation is zero”
Warning message in evaluate_target_prediction(setting, ligand_target_matrix, ligands_position):
“all target gene probability score predictions have same value”
Warning message in cor(prediction, response):
“the standard deviation is zero”
Warning message in cor(prediction, response, method = "s"):
“the standard deviation is zero”
Warning message in evaluate_target_prediction(setting, ligand_target_matrix, ligands_position):
“all target gene probability score predictions have same value”
Warn

Warning message in cor(prediction, response):
“the standard deviation is zero”
Warning message in cor(prediction, response, method = "s"):
“the standard deviation is zero”
Warning message in evaluate_target_prediction(setting, ligand_target_matrix, ligands_position):
“all target gene probability score predictions have same value”
Warning message in cor(prediction, response):
“the standard deviation is zero”
Warning message in cor(prediction, response, method = "s"):
“the standard deviation is zero”
Warning message in evaluate_target_prediction(setting, ligand_target_matrix, ligands_position):
“all target gene probability score predictions have same value”
Warning message in cor(prediction, response):
“the standard deviation is zero”
Warning message in cor(prediction, response, method = "s"):
“the standard deviation is zero”
Warning message in evaluate_target_prediction(setting, ligand_target_matrix, ligands_position):
“all target gene probability score predictions have same value”
Warn

Warning message in cor(prediction, response):
“the standard deviation is zero”
Warning message in cor(prediction, response, method = "s"):
“the standard deviation is zero”
Warning message in evaluate_target_prediction(setting, ligand_target_matrix, ligands_position):
“all target gene probability score predictions have same value”
Warning message in cor(prediction, response):
“the standard deviation is zero”
Warning message in cor(prediction, response, method = "s"):
“the standard deviation is zero”
Warning message in evaluate_target_prediction(setting, ligand_target_matrix, ligands_position):
“all target gene probability score predictions have same value”
Warning message in cor(prediction, response):
“the standard deviation is zero”
Warning message in cor(prediction, response, method = "s"):
“the standard deviation is zero”
Warning message in evaluate_target_prediction(setting, ligand_target_matrix, ligands_position):
“all target gene probability score predictions have same value”
Warn

Warning message in cor(prediction, response):
“the standard deviation is zero”
Warning message in cor(prediction, response, method = "s"):
“the standard deviation is zero”


In [48]:
matrix_result1 <- Reduce(
  
  # The `Reduce()` function takes two arguments: a function and a list.
  # In this case, the function is an anonymous function defined using the `function()` keyword.
  # This function takes two arguments `x` and `y` and performs a full join between them using the `full_join()` function from the `dplyr` package.
  # The `by = "interaction"` argument specifies that the join should be performed on the "interaction" column.
  function(x, y) full_join(x, y, by = "interaction_ID"), 
  
  # The second argument to the `Reduce()` function is a list called `results`.
  # This list contains data frames that need to be joined together.
  results
)

In [49]:
# Define a variable called `result` that will hold the output of the Reduce function
matrix_result <- Reduce(
  
  # The `Reduce()` function takes two arguments: a function and a list.
  # In this case, the function is an anonymous function defined using the `function()` keyword.
  # This function takes two arguments `x` and `y` and performs a full join between them using the `full_join()` function from the `dplyr` package.
  # The `by = "interaction"` argument specifies that the join should be performed on the "interaction" column.
  function(x, y) full_join(x, y, by = "interaction_ID"), 
  
  # The second argument to the `Reduce()` function is a list called `results`.
  # This list contains data frames that need to be joined together.
  results
)


In [50]:
# Example list of strings
strings <- matrix_result$interaction_ID

# Initialize empty vectors for each column
sender_celltype <- c()
sender_gene <- c()
receiver_celltype <- c()
receiver_gene <- c()

# Loop through each string and split it
for (string in strings) {
  parts <- strsplit(string, "_")
  
  # Split the sender part
  sender_parts <- strsplit(parts[[1]][1], ":")
  sender_celltype <- c(sender_celltype, sender_parts[[1]][1])
  sender_gene <- c(sender_gene, sender_parts[[1]][2])
  
  # Split the receiver part
  receiver_parts <- strsplit(parts[[1]][2], ":")
  receiver_celltype <- c(receiver_celltype, receiver_parts[[1]][1])
  receiver_gene <- c(receiver_gene, receiver_parts[[1]][2])
  interaction_ID <- c(string)
}

# Create a dataframe with the splitted values
df <- data.frame(
  sender_celltype = sender_celltype,
  sender_gene = sender_gene,
  receiver_celltype = receiver_celltype,
  receiver_gene = receiver_gene
)



In [51]:
# add interaction ID at the first column
df <- tibble::add_column(df, interaction_ID = matrix_result$interaction_ID, .before = "sender_celltype")

In [52]:
# Create interaction annotation df with log2FC values
for (row in 1:nrow(df)){
    each_row <- df[row,]
    sender_cell <- each_row$sender_celltype
    sender_gene <- each_row$sender_gene
    receiver_cell <- each_row$receiver_celltype
    receiver_gene <- each_row$receiver_gene
    
    DE_ligand_cell_type <- DE_tables[[sender_cell]]
    DE_receptor_cell_type <- DE_tables[[receiver_cell]]
    
    ligand_log2FC <- DE_ligand_cell_type[DE_ligand_cell_type$gene==sender_gene,]$avg_log2FC
    
    ligand_p_val_adj <- DE_ligand_cell_type[DE_ligand_cell_type$gene==sender_gene,]$p_val_adj
    
    receptor_log2FC <- DE_receptor_cell_type[DE_receptor_cell_type$gene==receiver_gene,]$avg_log2FC
    
    receptor_p_val_adj <- DE_receptor_cell_type[DE_receptor_cell_type$gene==receiver_gene,]$p_val_adj

    if (length(ligand_log2FC) == 0) {
        df[row,"ligand_log2FC"] <- NA
        df[row,"ligand_p_val_adj"] <- NA
    } else {
        df[row,"ligand_log2FC"] <- ligand_log2FC
        df[row,"ligand_p_val_adj"] <- ligand_p_val_adj
    }
    
    if (length(receptor_log2FC) == 0) {
        df[row,"receptor_log2FC"] <- NA
        df[row,"receptor_p_val_adj"] <- NA
    } else {
        df[row,"receptor_log2FC"] <- receptor_log2FC
        df[row,"receptor_p_val_adj"] <- receptor_p_val_adj
    }
}


In [53]:
#see where both components (sender/receiever) have log2FC value
df[complete.cases(df$ligand_log2FC, df$receptor_log2FC), ]

,interaction_ID,sender_celltype,sender_gene,receiver_celltype,receiver_gene,ligand_log2FC,ligand_p_val_adj,receptor_log2FC,receptor_p_val_adj
,<chr>,<chr>,<chr>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>
1,Mono:ACKR3_Mono:CXCR4,Mono,ACKR3,Mono,CXCR4,0.3832868,0.000000e+00,0.4670962,0.000000e+00
16,Mono:CD9_Mono:CD36,Mono,CD9,Mono,CD36,0.5042017,0.000000e+00,0.3866941,0.000000e+00
25,Mono:LILRB4_Mono:LAIR1,Mono,LILRB4,Mono,LAIR1,0.4668692,0.000000e+00,0.3156081,0.000000e+00
85,Gran:S100A12_Mono:CD36,Gran,S100A12,Mono,CD36,-0.4809041,1.744210e-45,0.3866941,0.000000e+00
87,Gran:S100A8_Mono:CD36,Gran,S100A8,Mono,CD36,-0.5741033,1.671713e-71,0.3866941,0.000000e+00
88,Gran:S100A8_Mono:CD68,Gran,S100A8,Mono,CD68,-0.5741033,1.671713e-71,0.3156466,0.000000e+00
92,Gran:S100A9_Mono:CD36,Gran,S100A9,Mono,CD36,-0.5467334,7.080928e-46,0.3866941,0.000000e+00
93,Gran:S100A9_Mono:CD68,Gran,S100A9,Mono,CD68,-0.5467334,7.080928e-46,0.3156466,0.000000e+00
325,B:MS4A1_Mono:CD53,B,MS4A1,Mono,CD53,-0.3329830,1.523841e-52,0.3318525,1.465202e-295


In [54]:
length(unique(lr_network$from))

[1] 1521

In [55]:
for (gene in df$sender_gene){
    if (!gene %in% lr_network$from){
        print(gene)
    }
}

In [56]:
df[!complete.cases(df$ligand_log2FC, df$receptor_log2FC), ]

,interaction_ID,sender_celltype,sender_gene,receiver_celltype,receiver_gene,ligand_log2FC,ligand_p_val_adj,receptor_log2FC,receptor_p_val_adj
,<chr>,<chr>,<chr>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>
2,Mono:ALOX5AP_Mono:ALOX5,Mono,ALOX5AP,Mono,ALOX5,0.3453332,4.466068e-239,NA,NA
3,Mono:AREG_Mono:ICAM1,Mono,AREG,Mono,ICAM1,NA,NA,NA,NA
4,Mono:CASP8_Mono:FAS,Mono,CASP8,Mono,FAS,NA,NA,NA,NA
5,Mono:CCL3L1_Mono:CCR1,Mono,CCL3L1,Mono,CCR1,NA,NA,NA,NA
6,Mono:CCR1_Mono:MBP,Mono,CCR1,Mono,MBP,NA,NA,NA,NA
7,Mono:CCR1_Mono:PRNP,Mono,CCR1,Mono,PRNP,NA,NA,NA,NA
8,Mono:CD14_Mono:ITGA4,Mono,CD14,Mono,ITGA4,NA,NA,NA,NA
9,Mono:CD14_Mono:ITGB1,Mono,CD14,Mono,ITGB1,NA,NA,NA,NA
10,Mono:CD14_Mono:ITGB2,Mono,CD14,Mono,ITGB2,NA,NA,NA,NA


In [58]:
write.csv(matrix_result, paste0(output_dir,"NicheNet_scores.csv"))

In [59]:
write.csv(df, paste0(output_dir,"NicheNet_anno_interactions.csv"))

In [60]:
nn_interactions <- list()

In [63]:
nn_interactions$weights <- matrix_result
nn_interactions$anno_interactions <- df

In [64]:
# reassign directions for NN
nn_interactions$anno_interactions$direction_lig <- NA
nn_interactions$anno_interactions$direction_rec <- NA

nn_interactions$anno_interactions$direction_lig[!is.na(nn_interactions$anno_interactions$ligand_log2FC) & (nn_interactions$anno_interactions$ligand_log2FC < 0)] <- "down"
nn_interactions$anno_interactions$direction_lig[!is.na(nn_interactions$anno_interactions$ligand_log2FC) & (nn_interactions$anno_interactions$ligand_log2FC > 0)] <- "up"

nn_interactions$anno_interactions$direction_rec[!is.na(nn_interactions$anno_interactions$receptor_log2FC) & (nn_interactions$anno_interactions$receptor_log2FC < 0)] <- "down"
nn_interactions$anno_interactions$direction_rec[!is.na(nn_interactions$anno_interactions$receptor_log2FC) & (nn_interactions$anno_interactions$receptor_log2FC > 0)] <- "up"

nn_interactions$anno_interactions$direction_lig_rec <- paste(nn_interactions$anno_interactions$direction_lig
                                            ,nn_interactions$anno_interactions$direction_rec
                                            ,sep = "_")
idx_down <- (nn_interactions$anno_interactions$direction_lig_rec == "down_NA") | (
    nn_interactions$anno_interactions$direction_lig_rec == "NA_down") | (
    nn_interactions$anno_interactions$direction_lig_rec == "down_down")
idx_up <- (nn_interactions$anno_interactions$direction_lig_rec == "up_NA") | (
    nn_interactions$anno_interactions$direction_lig_rec == "NA_up") | (
    nn_interactions$anno_interactions$direction_lig_rec == "up_up")

nn_interactions$anno_interactions$direction <- NA
nn_interactions$anno_interactions$direction[idx_down] <- "down"
nn_interactions$anno_interactions$direction[idx_up] <- "up"
nn_interactions$anno_interactions$direction[!(idx_down | idx_up)] <- "ambigous"

In [66]:
save(nn_interactions, file="outs/nn_interactions.RData")